In [45]:
import json 
import pandas as pd
import re
import nltk
from nltk.util import ngrams
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
quadgram_measures = nltk.collocations.QuadgramAssocMeasures()

#Strategy 1 scan after the word won:
df = pd.read_json("gg2013.json/gg2013.json")
df2 = df[df['text'].str.contains(" won ") | df['text'].str.contains(" Won ")]
tweets = df2['text'].tolist()
keywords = []
regexp = re.compile(r'[!?.]+(?=$|\s)')

#Parse Keywords
for tweet in tweets:
    words = tweet.lower().split()
    index_won = words.index("won")
    if words[index_won + 1] == "best":
        keyword = "best "
        curr_index = index_won + 2
        while curr_index != len(words):
            if regexp.search(words[curr_index]):
                keyword += words[curr_index]
                break
            keyword += words[curr_index] + " "
            curr_index += 1
        keywords.append(keyword)

#Analyze common phrase occurences
bigrams = []
for item in keywords:
    tokens = nltk.word_tokenize(item)
    grams = ngrams(tokens, 2)
    for gram in grams:
        bigrams.append(gram)
finder = BigramCollocationFinder.from_words(bigrams)
finder.apply_freq_filter(3)
#ignored_words = nltk.corpus.stopwords.words('english')
#finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
finder.nbest(bigram_measures.raw_freq, 30)

[(('!', '!'), ('!', '!')),
 (('best', 'original'), ('original', 'song')),
 (('the', 'golden'), ('golden', 'globes')),
 (('at', 'the'), ('the', 'golden')),
 (('best', 'director'), ('director', '.')),
 (('best', 'supporting'), ('supporting', 'actress')),
 (('best', 'director'), ('director', 'at')),
 (('director', 'at'), ('at', 'the')),
 (('director', '.'), ('best', 'director')),
 (('golden', 'globes'), ('globes', '.')),
 (('original', 'song'), ('song', 'for')),
 (('the', '#'), ('#', 'goldenglobes')),
 (('best', 'director'), ('director', 'for')),
 (('best', 'motion'), ('motion', 'picture')),
 (('actor', 'in'), ('in', 'a')),
 (('actress', 'in'), ('in', 'a')),
 (('best', 'supporting'), ('supporting', 'actor')),
 ((',', 'drama'), ('drama', '...')),
 (('a', 'motion'), ('motion', 'picture')),
 (('actress', ','), (',', 'drama')),
 (('best', 'actor'), ('actor', 'in')),
 (('best', 'actress'), ('actress', ',')),
 (('#', 'goldenglobes'), ('best', 'original')),
 (('at', 'the'), ('the', '#')),
 (('gl